In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling as pp

# I skip EDA procces becouse it was done in R

In [3]:
data = pd.read_csv(r'd:\3kurs1sem\semestrone\JUPYTER\Fractal\COURSE3\GroupAssigment\data\DataPreproccesed.csv')

In [4]:
pp.ProfileReport(data)

Number of variables,57
Number of observations,4410
Total Missing (%),0.0%
Total size in memory,1.9 MiB
Average record size in memory,456.0 B
Numeric,12
Categorical,0
Boolean,45
Date,0
Text (Unique),0
Rejected,0


In [5]:
X = data.drop('Attrition', axis=1)

In [8]:
y = data.Attrition

# Extra trees can find imortant features

In [11]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesRegressor
# load the iris datasets
model = ExtraTreesRegressor()
model.fit(X, y)
# display the relative importance of each attribute
pd.Series(model.feature_importances_).sort_values(ascending=False).head()

D:\3kurs1sem\IDEs\envs\PySpark\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10    0.080807
0     0.051551
5     0.047430
7     0.043541
39    0.034340
dtype: float64

# Top 5 features 

In [12]:
X.iloc[:,[10,0,5,7,39]].head()

,AvgWorkTime,Age,TotalWorkingYears,YearsAtCompany,MaritalStatus.xSingle
0,-0.25,1.54,-1.39,-1.15,0
1,0.01,-0.65,-0.70,-0.32,1
2,-0.52,-0.54,-0.84,-0.32,0
3,-0.38,0.12,0.27,0.30,0
4,0.23,-0.54,-0.29,-0.11,1


# Lets look in coefficients in Lasso regularizated model

In [20]:
from sklearn.linear_model import Lasso
ll = Lasso(alpha=0.0001)
ll.fit(X,y)
pd.Series(ll.coef_).sort_values(ascending=False).head()

12    0.152887
39    0.124472
43    0.107278
34    0.093975
10    0.074066
dtype: float64

# Top 5 features 

In [21]:
X.iloc[:,[12,39,43,34,10]].head()

,BusinessTravel.xTravel_Frequently,MaritalStatus.xSingle,EnvironmentSatisfaction.xLow,JobRole.xResearch.Director,AvgWorkTime
0,0,0,0,0,-0.25
1,1,1,0,0,0.01
2,1,0,0,0,-0.52
3,0,0,0,0,-0.38
4,0,1,0,0,0.23


# RandomForest also can find imortant features

In [22]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42).fit(X, y)
pd.Series(rf.feature_importances_).sort_values(ascending=False).head()

D:\3kurs1sem\IDEs\envs\PySpark\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10    0.120093
5     0.083233
0     0.082449
2     0.068486
1     0.050100
dtype: float64

# Top 5 features 

In [23]:
X.iloc[:,[10,5,0,2,1]].head()

,AvgWorkTime,TotalWorkingYears,Age,MonthlyIncome,DistanceFromHome
0,-0.25,-1.39,1.54,1.40,-0.39
1,0.01,-0.70,-0.65,-0.49,0.10
2,-0.52,-0.84,-0.54,2.72,0.96
3,-0.38,0.27,0.12,0.39,-0.89
4,0.23,-0.29,-0.54,-0.88,0.10
